In [154]:
import pandas as pd
import numpy as np

In [155]:
FILE_PATH = 'day_2_data.txt'

In [156]:
raw_df = pd.read_csv(FILE_PATH, delimiter='\t', header=None)
raw_df.columns = ['text']

In [157]:
df = (
    raw_df
    .assign(**{
        'id': lambda x: x['text'].str.split(r':', expand=True)[0].str.replace('Game ', ''),
        'value': lambda x: x['text'].str.split(r':', expand=True)[1].str.strip(),
    })
    .drop(columns=['text'])
)
subsets = df['value'].str.split(r';', expand=True).fillna('')
subsets = subsets.add_prefix('subset_', axis='columns')
df = pd.concat([df, subsets], axis=1).drop(columns=['value'])
df.head()

,id,subset_0,subset_1,subset_2,subset_3,subset_4,subset_5
0,1,"4 red, 5 blue, 9 green","7 green, 7 blue, 3 red","16 red, 7 blue, 3 green","11 green, 11 blue, 6 red","12 red, 14 blue",
1,2,"12 blue, 11 green, 3 red","6 blue, 5 green, 7 red","5 red, 11 blue","2 blue, 8 green",,
2,3,"8 blue, 5 green, 2 red","5 blue, 5 green, 7 red","7 blue, 1 green, 7 red","8 green, 14 blue, 7 red","8 green, 14 blue","8 blue, 2 green, 8 red"
3,4,"3 red, 14 blue, 15 green","1 red, 11 green, 14 blue","14 green, 17 blue",,,
4,5,"11 green, 2 red, 10 blue","16 green, 8 blue","2 blue, 6 green, 1 red","14 blue, 2 red, 16 green","3 blue, 18 green","1 red, 10 blue, 3 green"


In [158]:
subset_cols = df.filter(regex='subset').columns
dict_colors = {
    'red': 12,
    'green': 13,
    'blue': 14,
}

for _, key_ in enumerate(subset_cols):

    for color, condition in zip(dict_colors.keys(), dict_colors.values()):
        df = (
            df
            .assign(**{
                f'{color}_{key_}': lambda x: x[key_].str.extract(fr'(\d+) {color}').fillna(0).astype('int64'),
            })
        )

df = df.filter(regex='id|_subset_.*').sort_index(axis=1)
df.head()

,blue_subset_0,blue_subset_1,blue_subset_2,blue_subset_3,blue_subset_4,blue_subset_5,green_subset_0,green_subset_1,green_subset_2,green_subset_3,green_subset_4,green_subset_5,id,red_subset_0,red_subset_1,red_subset_2,red_subset_3,red_subset_4,red_subset_5
0,5,7,7,11,14,0,9,7,3,11,0,0,1,4,3,16,6,12,0
1,12,6,11,2,0,0,11,5,0,8,0,0,2,3,7,5,0,0,0
2,8,5,7,14,14,8,5,5,1,8,8,2,3,2,7,7,7,0,8
3,14,14,17,0,0,0,15,11,14,0,0,0,4,3,1,0,0,0,0
4,10,8,2,14,3,10,11,16,6,16,18,3,5,2,0,1,2,0,1


In [160]:
# Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

for color, value in dict_colors.items():

    print(color, value)

    df = (
        df
        .assign(**{
            f'{color}_flag': lambda x: x.filter(regex=f'{color}').le(value).all(axis=1),
            # 'possible': lambda x: x.filter(regex='flag').all(axis=1),
        })
        .sort_index(axis=1)
    )
df['possible'] = df.filter(regex='flag').all(axis=1)
df.head()

red 12
green 13
blue 14


,blue_flag,blue_subset_0,blue_subset_1,blue_subset_2,blue_subset_3,blue_subset_4,blue_subset_5,green_flag,green_subset_0,green_subset_1,...,green_subset_5,id,red_flag,red_subset_0,red_subset_1,red_subset_2,red_subset_3,red_subset_4,red_subset_5,possible
0,True,5,7,7,11,14,0,True,9,7,...,0,1,False,4,3,16,6,12,0,False
1,True,12,6,11,2,0,0,True,11,5,...,0,2,True,3,7,5,0,0,0,True
2,True,8,5,7,14,14,8,True,5,5,...,2,3,True,2,7,7,7,0,8,True
3,False,14,14,17,0,0,0,False,15,11,...,0,4,True,3,1,0,0,0,0,False
4,True,10,8,2,14,3,10,False,11,16,...,3,5,True,2,0,1,2,0,1,False


In [162]:
print(df.loc[df['possible'] == True, ['id']].sum())

id    2381112151619212226282931353638404243465051525...
dtype: object


In [165]:
# Answer
from collections import Counter
from functools import reduce
from operator import mul, or_

thres = Counter({"red":12, "green":13, "blue":14})

tot_1 = 0
tot_2 = 0
with open("day_2_data.txt", "r") as f:
    for game in f:
        game_id, draws = game.strip().split(": ")
        game_id = int(game_id.split(" ")[1])
        draws = [[c.split(" ") for c in d.split(", ")] for d in draws.split("; ")]
        draws = [Counter({c[1]:int(c[0]) for c in d}) for d in draws]
        tot_1 += all(d<=thres for d in draws) * game_id
        tot_2 += reduce(mul, reduce(or_, draws).values())
        

print(1, tot_1)
print(2, tot_2)

1 2795
2 75561
